In [0]:
#import keras 
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.optimizers import SGD, Adam
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, LSTM, Dropout, GaussianNoise, BatchNormalization, Input, concatenate, Activation
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.python.keras.regularizers import l1, l2, l1_l2

# from keras.optimizers import  SGD, Adam
# from keras.models import Sequential, Model
# from keras.layers import Dense, LSTM, Dropout, GaussianNoise, BatchNormalization, Input, concatenate, Activation
# from keras.callbacks import ReduceLROnPlateau, EarlyStopping
# from keras.regularizers import l1, l2, l1_l2
keras.backend.clear_session()

In [0]:
print(tf.__version__)

2.1.0

In [0]:
def build_lstm_model(input_shape,output_shape):
#     model = Sequential()
#     model.add(LSTM(256, return_sequences=True,input_shape=input_shape))
#       #model.add(BatchNormalization())
#       #model.add(Activation('tanh'))
#     model.add(Dropout(0.1, seed=0)) #Dropout(0.1)) noise_shape=None, seed=None
#     model.add(LSTM(256, return_sequences=True))
#       #model.add(BatchNormalization())
#       #model.add(Activation('tanh'))
#       #model.add(Dropout(0.1))
#     model.add(LSTM(256, kernel_regularizer=l1_l2(0.001,0.001)))
#       #model.add(BatchNormalization())
#       #model.add(Activation('tanh'))
#       #model.add(Dropout(0.1))
#     model.add(Dense(output_shape,activation='linear'))
    
    #update to have UQ
    
#     lstm1 = LSTM(256, return_sequences=True,input_shape=input_shape)
#     dropout1 = Dropout(0.1, seed=0)
#     lstm2 = LSTM(256, return_sequences=True)
#     lstm3 = LSTM(256, kernel_regularizer=l1_l2(0.001,0.001))
#     dense = Dense(output_shape,activation='linear')
    
    inputs = keras.Input(shape=input_shape)
    x = LSTM(256, return_sequences=True,input_shape=input_shape)(inputs, training=True)
    x = Dropout(0.1, seed = 0)(x, training=True)
    x = LSTM(256, return_sequences=True)(x, training = True)
    x = LSTM(256, kernel_regularizer=l1_l2(0.001,0.001))(x, training = True)
    outputs = Dense(output_shape,activation='linear')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
#     inputs = keras.Input(shape=input_shape)
#     x = keras.layers.LSTM(512,recurrent_dropout=dropout_value)(inputs, training=True)
#     x = keras.layers.Dropout(dropout_value)(x, training=True)
#     x = keras.layers.RepeatVector(n=X_train.shape[1])(x)
#     x = keras.layers.LSTM(units=512,activation='tanh',recurrent_dropout=0.1, return_sequences=True)(x)
#     x = keras.layers.Dropout(dropout_value)(x, training=True)
#     outputs = keras.layers.TimeDistributed(
#                  keras.layers.Dense(units=X_train.shape[2])
#              )(x)
#     model = keras.Model(inputs, outputs)
    
    return model

def build_complex_model(injector_shape,booster_shape):
    # Define the tensors for the two input images
    injector_input = Input(injector_shape)
    booster_input  = Input(booster_shape)
    
    ## 
    injector_model = build_lstm_model(input_shape=injector_shape)
    
    ##
    booster_model = build_lstm_model(input_shape=booster_shape)

    ##
    injector_output = injector_model(injector_input)
    booster_output = booster_model(booster_input)
    
    ##
    complex_output = concatenate([booster_output,injector_output])
    
    ##
    complext_net = Model(inputs=[injector_input,booster_input],outputs=complex_output)

    return complext_net

def train_lstm_model(in_shape,out_shape,x,y,epochs=5,batch_size=256):
    ## Make sure that the session is cleared
    keras.backend.clear_session()
    ## Start training
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
    #with strategy.scope():
    model = build_lstm_model(input_shape=in_shape,output_shape=out_shape)
    opt = Adam(lr=1e-2,clipnorm=1.0, clipvalue=0.5)
    #opt = SGD(lr=1e-2,nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=opt)
    model.summary()
    from keras.callbacks import ReduceLROnPlateau, EarlyStopping
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.85, patience=5, min_lr=1e-6,verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss',
                                   min_delta=0, patience=10, verbose=1, mode='auto',
                                   baseline=None, restore_best_weights=False)
    
    history = model.fit(x, y, epochs=epochs, batch_size=batch_size, validation_split=0.2,
                        callbacks=[reduce_lr,early_stopping], verbose=2, shuffle=True)
    return history, model